# Roman 72-Day Persistence Study


## Step 1: Create Catalog

In [ ]:
from astropy.table import Table
import astropy.units as u

MAG = 10
RA, DEC = 80.0, 30.0

catalog = Table()
catalog['ra'], catalog['dec'] = [RA], [DEC]
catalog['F146'], catalog['type'] = [MAG], ['PSF']
catalog['ra'].unit = u.deg
catalog['dec'].unit = u.deg

catalog.write(f'star{MAG}.ecsv', format='ascii.ecsv', overwrite=True)
print(f' Catalog: star{MAG}.ecsv')

## Step 2: Run Optimized Pipeline

In [ ]:
# ============================================================================
# FULLY SELF-CONTAINED SPACE-EFFICIENT PIPELINE
# ============================================================================
# This cell contains EVERYTHING needed - no other cells required!
# Run ONLY this cell to process all 8,640 exposures
# ============================================================================

# ============================================================================
# IMPORTS
# ============================================================================
import argparse
import asdf
import warnings
import copy
from datetime import datetime, timedelta
import gc
import glob
import json
import logging
import matplotlib.pyplot as plt
import numpy as np
import os
from pathlib import Path
import pandas as pd
import time
from typing import Dict, List, Tuple, Optional

import astropy
from astropy.time import Time
import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.stats import SigmaClip, sigma_clipped_stats
from astropy.table import Table
from astropy.visualization import simple_norm

import crds
import galsim
import romanisim
from romanisim import gaia, bandpass, catalog, log, wcs, persistence, parameters, ris_make_utils as ris
from romanisim.wcs import get_wcs
import romancal
from romancal.pipeline import ExposurePipeline
import roman_datamodels as rdm
from roman_datamodels.dqflags import pixel as dqflags

from photutils.aperture import CircularAperture, CircularAnnulus, aperture_photometry, ApertureStats

logging.getLogger().setLevel(logging.WARNING)


# Suppress expected ASDF warning when saving NaN values (saturated pixel replacement)
warnings.filterwarnings('ignore', message='invalid value encountered in cast', category=RuntimeWarning)

print(" All imports loaded")

# ============================================================================
# GLOBAL CONFIGURATION
# ============================================================================

# Observation parameters
OBSERVATION_PERIOD_DAYS = 72
CADENCE_MINUTES = 12
CADENCE_SECONDS = CADENCE_MINUTES * 60  # 720 seconds
TOTAL_TIME_SECONDS = OBSERVATION_PERIOD_DAYS * 24 * 3600
N_EXPOSURES = int(TOTAL_TIME_SECONDS / CADENCE_SECONDS)  # 8,640 exposures

# Simulation parameters
SCA = 1
FILTER = 'F146'
FIELD_CENTER = [80, 30]  # RA, Dec in degrees
MA_TABLE = 3
SEED = 42
BASE_TIME = '2026-10-15T00:00:00'
MAG = 10
CATALOG_FILE_NAME = f'star{MAG}.ecsv'

# PERSISTENCE MODELING: Each exposure uses the previous L1 file to model
# detector persistence effects. The pipeline automatically passes the previous
# L1 file to romanisim for accurate persistence simulation across the time series.

# Directories
RESULTS_DIR = './analysis_results'
TEMP_L1_DIR = './temp_l1_processing'
CHECKPOINT_FILE = './checkpoint.json'

# Output CSV files
L1_METRICS_CSV = os.path.join(RESULTS_DIR, 'l1_metrics.csv')
PHOTOMETRY_CSV = os.path.join(RESULTS_DIR, f'Photometry_Mag_{MAG}.csv')
PERSISTENCE_CSV = os.path.join(RESULTS_DIR, 'persistence.csv')

# Source positions for photometry
# source_positions will be set dynamically after auto-detection
source_positions = []  # Will be populated during L2 processing

# Read noise by detector
sca_read_noise = {
    'WFI01': 17.49, 'WFI02': 14.20, 'WFI03': 14.70, 'WFI04': 15.76,
    'WFI05': 13.64, 'WFI06': 13.65, 'WFI07': 13.21, 'WFI08': 12.49,
    'WFI09': 12.39, 'WFI10': 15.75, 'WFI11': 13.94, 'WFI12': 13.73,
    'WFI13': 16.17, 'WFI14': 13.46, 'WFI15': 13.46, 'WFI16': 13.49,
    'WFI17': 12.64, 'WFI18': 12.42,
}

print(" Global configuration loaded")

# ============================================================================
# PHOTOMETRIC CALIBRATION CONSTANTS
# ============================================================================

# Roman WFI F146 filter parameters
PIXEL_SCALE = 0.11  # arcsec/pixel
PHOTFLAM = 3.0151335e-20  # erg/cm^2/Angstrom/electron for F146
PHOTPLAM = 14539.0  # Pivot wavelength in Angstroms for F146
ZERO_POINT_AB = 28.08  # AB magnitude zero point for F146

# Conversion factor: DN to Jy
COUNTS_TO_JY = PIXEL_SCALE**2 * PHOTFLAM * PHOTPLAM / 3.34e4

print(" Photometric constants loaded")

# ============================================================================
# INTERMEDIATE DATA EXTRACTION CONFIGURATION
# ============================================================================

# Enable/disable extraction of intermediate file data
EXTRACT_JUMPS = False         # DISABLED for speed
EXTRACT_WCS = False           # DISABLED for speed
EXTRACT_RAMP_QUALITY = False  # Ramp fit quality (Optional)
EXTRACT_DQ_FLAGS = False      # DQ flags summary (Optional)

print(" Intermediate data extraction configured")
print(f"  Jump detection: {EXTRACT_JUMPS}")
print(f"  WCS astrometry: {EXTRACT_WCS}")
print(f"  Ramp quality: {EXTRACT_RAMP_QUALITY}")
print(f"  DQ flags: {EXTRACT_DQ_FLAGS}")



# ============================================================================
# ROMAN L1 GENERATION FUNCTION
# ============================================================================

def run_romanisim_simulationapi(
    catalog_file,
    filter_name='F146',
    sca=1,
    level=1,
    exposure_params=None,
    pointing=(270.0, -30.0),
    date='2026-06-15T12:00:00',
    output_directory='./L1_simulation_files',
    output_suffix="",
    previous_file=None,
    usecrds=True
):
    """Generate Roman L1 simulation file using romanisim."""
    
    detector = f'WFI{sca:02d}'
    output_filename = f"{output_directory}/roman_sim_{filter_name}_{detector}_L{level}{output_suffix}.asdf"
    
    # Get parameters from exposure_params dict
    ma_table_number = exposure_params.get('ma_table_number', 3) if exposure_params else 3
    seed = exposure_params.get('seed', 7) if exposure_params else 7
    
    # Set up argument parser (no add_args - not needed)
    parser = argparse.ArgumentParser()
    parser.set_defaults(
        usecrds=usecrds, 
        stpsf=True, 
        level=level, 
        filename=output_filename, 
        drop_extra_dq=True, 
        sca=sca, 
        bandpass=filter_name, 
        pretend_spectral=None
    )
    
    args = parser.parse_args([])
    
    # Set reference files to None for CRDS
    for k in parameters.reference_data:
        parameters.reference_data[k] = None
    
    # Set Galsim RNG object
    rng = galsim.UniformDeviate(seed)
    
    # Set persistence information
    if previous_file and os.path.exists(previous_file):
        persist = persistence.Persistence.read(previous_file)
    else:
        persist = persistence.Persistence()
    
    # Set metadata using romanisim's helper
    metadata = ris.set_metadata(
        date=date, 
        bandpass=filter_name, 
        sca=sca, 
        ma_table_number=ma_table_number, 
        usecrds=usecrds
    )
    
    # Update the WCS info
    wcs.fill_in_parameters(
        metadata, 
        SkyCoord(pointing[0], pointing[1], unit='deg', frame='icrs'), 
        boresight=False, 
        pa_aper=0.0
    )
    
    # Load catalog
    cat = astropy.table.Table.read(catalog_file)
    
    # Run the simulation
    ris.simulate_image_file(args, metadata, cat, rng, persist)
    
    return output_filename

print(" L1 generation function loaded")

# ============================================================================
# ROMAN L1 RESULTANT ANALYZER CLASS
# ============================================================================



# ======================================================================
# MASTER NOTEBOOK INTEGRATION
# ======================================================================


class RomanL1ResultantAnalyzer:
    """
    Analyzer for individual resultants in Roman L1 files with proper MA table handling.
    Full implementation from Master notebook.
    """
    
    def __init__(self, l1_file_path: str):
        """
        Initialize with L1 file path.
        
        Parameters:
        -----------
        l1_file_path : str
            Path to L1 ASDF file
        """
        self.file_path = l1_file_path
        self.model = None
        
        # Data containers
        self.ramp_data = None  # (resultants, y, x)
        self.ref_pixels = None
        self.resultants = {}
        self.exposure_metadata = {}
        
        # MA table and timing information
        self.ma_table_info = {}
        self.read_pattern = []
        self.resultant_times = None
        self.frame_times = None
        
        # Processed data
        self.slopes = None
        self.slope_errors = None
        self.corrected_ramp = None
        self.saturation_mask = None
        
        # Read noise per detector (actual Roman WFI values)
        # These are the measured read noise values for each WFI detector
        self.sca_read_noise = {
            'WFI01': 17.49, 'WFI02': 14.20, 'WFI03': 14.70, 'WFI04': 15.76,
            'WFI05': 13.64, 'WFI06': 13.65, 'WFI07': 13.21, 'WFI08': 12.49,
            'WFI09': 12.39, 'WFI10': 15.75, 'WFI11': 13.94, 'WFI12': 13.73,
            'WFI13': 16.17, 'WFI14': 13.46, 'WFI15': 13.46, 'WFI16': 13.49,
            'WFI17': 12.64, 'WFI18': 12.42,
        }
        
        # Exposure time from MA table
        self.exposure_time = None
    
    def load_l1_file(self):
        """Load the L1 file and extract comprehensive metadata."""
        print(f"Loading L1 file: {self.file_path}")
        
        self.model = rdm.open(str(self.file_path))
        self.ramp_data = self.model.data
        
        # Extract detailed exposure metadata
        meta = self.model.meta
        exposure = meta.exposure
        
        self.exposure_metadata = {
            'type': exposure.type,
            'start_time': exposure.start_time,
            'mid_time': exposure.mid_time,
            'end_time': exposure.end_time,
            'nresultants': exposure.nresultants,
            'frame_time': exposure.frame_time,
            'exposure_time': exposure.exposure_time,
            'effective_exposure_time': exposure.effective_exposure_time,
            'ma_table_name': exposure.ma_table_name,
            'ma_table_number': exposure.ma_table_number,
            'truncated': exposure.truncated,
            'data_problem': exposure.data_problem,
            'detector': meta.instrument.detector,
            'optical_element': meta.instrument.optical_element,
            'read_noise': self.sca_read_noise.get(meta.instrument.detector, 5.0)
        }
        
        # Store exposure time
        self.exposure_time = exposure.exposure_time
        
        # Extract the actual read pattern (MA table structure)
        self.read_pattern = exposure.read_pattern
        print(f"Found MA table read pattern with {len(self.read_pattern)} resultants")
        
        # Process the MA table information
        self._process_ma_table()
        
        print(f"Loaded ramp data: {self.ramp_data.shape}")
        print(f"MA table {self.exposure_metadata.get('ma_table_number', '?')}: "
              f"{len(self.read_pattern)} resultants")
    
    def _process_ma_table(self):
        """Process the MA table read pattern to understand timing."""
        frame_time = self.exposure_metadata['frame_time']
        
        # Calculate timing for each resultant based on the read pattern
        self.resultant_times = []
        self.frame_times = []
        
        for i, read_group in enumerate(self.read_pattern):
            # Each read_group is a list of frame numbers
            if len(read_group) > 0:
                last_frame_number = max(read_group)
                resultant_time = last_frame_number * frame_time
                
                self.resultant_times.append(resultant_time)
                self.frame_times.append(read_group)
                
                print(f"  Resultant {i}: frames {read_group}, time = {resultant_time:.2f}s")
        
        self.resultant_times = np.array(self.resultant_times)
        
        # Store MA table analysis
        self.ma_table_info = {
            'total_frames': max([max(group) for group in self.read_pattern if group]),
            'total_time': self.resultant_times[-1] if len(self.resultant_times) > 0 else 0,
            'resultant_count': len(self.read_pattern),
            'frame_time': frame_time
        }
        
        print(f"  MA table: {self.ma_table_info['total_frames']} frames, "
              f"{self.ma_table_info['total_time']:.2f}s total")

    def extract_all_resultants(self):
        """Extract individual resultant arrays with proper MA table context."""
        print("Extracting individual resultants with MA table context...")

        for i in range(self.exposure_metadata["nresultants"]):
            resultant_name = f"resultant_{i:02d}"
            self.resultants[resultant_name] = self.ramp_data[i, :, :].copy()
            
            # Get information about this resultant from MA table
            if i < len(self.read_pattern):
                read_group = self.read_pattern[i]
                n_frames_averaged = len(read_group) 
                resultant_time = self.resultant_times[i] if i < len(self.resultant_times) else 0
            else:
                n_frames_averaged = 1
                resultant_time = 0
            
            # Basic statistics
            valid_mask = np.isfinite(self.resultants[resultant_name])
            if np.any(valid_mask):
                mean_val = np.mean(self.resultants[resultant_name][valid_mask])
                median_val = np.median(self.resultants[resultant_name][valid_mask])
                print(f"  {resultant_name}: time={resultant_time:.2f}s, "
                           f"frames_avg={n_frames_averaged}, "
                           f"mean={mean_val:.1f} DN, median={median_val:.1f} DN")
        
        print(f"Extracted {len(self.resultants)} resultants")
        return self.resultants

    
    def casertano_weighted_fit(self, times, counts, read_noise=10.0):
        """
        Implement Casertano (2022) optimal weighting scheme.
        
        Adapts weighting based on signal-to-noise ratio:
        - Low S/N: Equal weighting (read noise dominated)
        - High S/N: Endpoints only (Poisson dominated)
        - Intermediate: Smooth transition
        """
        N = len(times)
        if N < 2:
            return 0.0, np.inf
        
        # Convert to float64 to prevent overflow
        counts = np.asarray(counts, dtype=np.float64)
        times = np.asarray(times, dtype=np.float64)
        
        # Calculate signal-to-noise ratio
        S_max = counts[-1] - counts[0]
        if S_max <= 0:
            S_max = np.abs(counts[-1] - counts[0]) + 1.0
        
        S = S_max / np.sqrt(read_noise**2 + S_max)
        
        # Calculate optimal weights based on S/N
        weights = self.calculate_casertano_weights(times, S, read_noise)
        
        # Apply weighted least squares
        W = weights
        t_bar = times - times[0]  # Relative times
        
        # Auxiliary sums (Casertano method)
        F0 = np.sum(W)
        F1 = np.sum(W * t_bar)
        F2 = np.sum(W * t_bar**2)
        
        D = F0 * F2 - F1**2
        if abs(D) < 1e-15:
            return 0.0, np.inf
        
        # Coefficients and slope calculation
        K = (F0 * t_bar - F1) / D
        slope = np.sum(K * counts)
        
        # Calculate uncertainty
        tau = times - times[0]
        V_S = np.sum(K**2 * tau) + np.sum([2 * K[i] * K[j] * t_bar[i] 
                                        for i in range(N) for j in range(i+1, N)])
        V_R = np.sum(K**2) * read_noise**2
        V_total = V_S * abs(slope) + V_R
        slope_error = np.sqrt(max(V_total, 0))
        
        return slope, slope_error
    
    def calculate_casertano_weights(self, times, S, read_noise):
        """
        Calculate optimal weights based on S/N ratio.
        
        Parameters:
        -----------
        S : float
            Signal-to-noise ratio
        
        Returns:
        --------
        weights : array
            Optimal weights for each resultant
        """
        N = len(times)
        
        # Low S/N: equal weighting (read noise dominated)
        if S < 0.1:
            return np.ones(N)
        
        # High S/N: first and last samples only (Poisson dominated)
        elif S > 10.0:
            weights = np.zeros(N)
            weights[0] = 1.0   # First
            weights[-1] = 1.0  # Last
            return weights
        
        # Intermediate S/N: smooth transition
        else:
            equal_weights = np.ones(N)
            endpoint_weights = np.zeros(N)
            endpoint_weights[0] = 1.0
            endpoint_weights[-1] = 1.0
            
            # Sigmoid transition parameter
            alpha = 1.0 / (1.0 + np.exp(-2 * (S - 2)))
            weights = (1 - alpha) * equal_weights + alpha * endpoint_weights
            
            return weights
    
    def reference_pixel_correction(self, method='median_subtract'):
        """Apply reference pixel correction using 4-pixel border."""
        # Convert to float64 for calculations
        self.corrected_ramp = self.ramp_data.astype(np.float64)
        
        if method == 'none':
            return
        
        for i in range(self.corrected_ramp.shape[0]):
            resultant = self.corrected_ramp[i, :, :]
            
            # Extract reference pixels from 4-pixel border
            ref_pixels = np.concatenate([
                resultant[:4, :].flatten(),      # Top 4 rows
                resultant[-4:, :].flatten(),     # Bottom 4 rows
                resultant[4:-4, :4].flatten(),   # Left 4 cols
                resultant[4:-4, -4:].flatten()   # Right 4 cols
            ])
            
            valid_ref = ref_pixels[np.isfinite(ref_pixels)]
            
            if len(valid_ref) > 0:
                if method == 'median_subtract':
                    correction = np.median(valid_ref)
                else:  # mean_subtract
                    correction = np.mean(valid_ref)
                
                # Apply correction to science region
                self.corrected_ramp[i, 4:-4, 4:-4] -= correction
    
    def fit_up_the_ramp_slopes(self, method='casertano', read_noise=10.0):
        """
        Fit slopes using Casertano optimal weighting.
        
        This is the Master notebook's full implementation.
        """
        data_to_fit = self.corrected_ramp if self.corrected_ramp is not None else self.ramp_data.astype(np.float64)
        ny, nx = data_to_fit.shape[1:]
        self.slopes = np.zeros((ny, nx))
        self.slope_errors = np.zeros((ny, nx))
        
        print(f"Fitting slopes with Casertano weighting (read_noise={read_noise:.1f} DN)...")
        
        for y in range(ny):
            for x in range(nx):
                times = self.resultant_times
                counts = data_to_fit[:, y, x]
                
                # Use Casertano optimal weighting
                slope, error = self.casertano_weighted_fit(times, counts, read_noise)
                
                self.slopes[y, x] = slope
                self.slope_errors[y, x] = error
        
        return self.slopes, self.slope_errors



def get_saturated_pixels_and_neighbors(data, threshold):
    """
    Get saturated pixels and their 8 neighbors (FAST VECTORIZED - MASTER LOGIC).
    
    Matches Master notebook behavior:
    - Finds core saturated pixels
    - Adds 8 neighbors (3×3 box)
    - Prints core count like Master
    
    Parameters:
    -----------
    data : np.ndarray
        2D image data
    threshold : float
        Saturation threshold in DN
    
    Returns:
    --------
    saturated_pixels : list of tuples
        List of (y, x) coordinates including neighbors
    """
    from scipy import ndimage
    
    # Find core saturated pixels (vectorized - FAST!)
    saturated_mask = data >= threshold
    pixel_count = np.sum(saturated_mask)  # Core count
    
    if pixel_count == 0:
        print(f'  Core saturated pixels: 0')
        print(f'  Total with neighbors: 0')
        return []
    
    # Dilate to include 8 neighbors (vectorized - FAST!)
    structure = np.array([[1, 1, 1],
                         [1, 1, 1],
                         [1, 1, 1]], dtype=bool)
    
    dilated_mask = ndimage.binary_dilation(saturated_mask, structure=structure)
    
    # Convert mask to list of coordinates
    coords = np.where(dilated_mask)
    saturated_pixels = list(zip(coords[0], coords[1]))
    
    # Print like Master notebook
    print(f'  Core saturated pixels: {pixel_count}')
    print(f'  Total with neighbors: {len(saturated_pixels)}')
    
    return saturated_pixels

def replace_saturated_pixels(data, indices, replacement_value=np.nan):
    """
    Replace saturated pixels with specified value.
    
    From Master notebook - typically replaces with NaN for clean photometry.
    
    Parameters:
    -----------
    data : 2D array
        Data to modify
    indices : list of tuples
        Pixel coordinates to replace
    replacement_value : float
        Value to use (default: np.nan)
    
    Returns:
    --------
    modified_data : 2D array
        Data with saturated pixels replaced
    """
    modified_data = data.copy()
    for index in indices:
        modified_data[index] = replacement_value
    
    return modified_data



def update_unsaturated_resultant(file_name, resultants_data):
    """
    Updates all resultants in the L1 file at once.
    
    From Master notebook - replaces saturated pixels in L1 BEFORE pipeline.
    
    Parameters:
    -----------
    file_name : str
        Path to the L1 ASDF file
    resultants_data : list of 2D arrays
        Modified resultant data, one for each resultant
    """
    with rdm.open(file_name) as model:
        new_model = model.copy()
        for i, data in enumerate(resultants_data):
            new_model.data[i, :, :] = data
        new_model.save(file_name.replace('.asdf', '_unsat.asdf'))
        return file_name.replace('.asdf', '_unsat.asdf')

def get_saturated_pixels_from_dq_with_neighbors(dq_array, saturation_threshold=65000):
    """
    Get saturated pixels from DQ flags OR threshold, with neighbors.
    
    Combines DQ flag method with threshold check for robustness.
    
    Parameters:
    -----------
    dq_array : 2D array
        DQ flags from L2
    saturation_threshold : float
        DN threshold for saturation (default 65000)
    
    Returns:
    --------
    coords : list of tuples
        Saturated pixel coordinates with neighbors
    n_core : int
        Core saturated pixels
    """
    from scipy import ndimage
    
    # Get saturated from DQ bit 1
    saturated_bit = 1
    saturated_mask = (dq_array & (1 << saturated_bit)) > 0
    
    n_core = np.sum(saturated_mask)
    
    # Dilate to include neighbors
    structure = np.ones((3, 3), dtype=bool)
    expanded_mask = ndimage.binary_dilation(saturated_mask, structure=structure)
    
    # Convert to coordinates
    coords = list(zip(*np.where(expanded_mask)))
    
    return coords, n_core



# ======================================================================
# MASTER NOTEBOOK HELPER FUNCTIONS
# ======================================================================

def get_asdf_file(file_path):
    """Open ASDF file."""
    return asdf.open(file_path, 'rb')

def get_roman_dm_file(file_path):
    """Open file with roman_datamodels."""
    with asdf.open(file_path, 'rb') as fb:
        f = rdm.open(fb).copy()
    return f

def get_metadata(file_path):
    """Extract metadata from file."""
    f = get_roman_dm_file(file_path)
    return f.meta

def get_data(file_path):
    """Extract data array from file."""
    f = get_roman_dm_file(file_path)
    return f.data

def get_data_quality_section(file_path):
    """Extract DQ array from file."""
    f = get_roman_dm_file(file_path)
    return f.dq

def get_list_of_asdf_files(directory):
    """Get list of ASDF files in directory, sorted by creation time."""
    dir_path = Path(directory)
    if not dir_path.is_dir():
        raise NotADirectoryError(f"{directory} is not a valid directory.")
    
    asdf_files = list(dir_path.glob('*.asdf'))
    asdf_files.sort(key=lambda x: os.path.getctime(x))
    return asdf_files

def plot_resultants_for_pixel(file, pixel_x, pixel_y):
    """Plot up-the-ramp samples for a specific pixel."""
    plt.figure(figsize=(6, 6), layout='tight')
    plt.title(f'Up-the-ramp samples for pixel {pixel_x}, {pixel_y}')
    plt.plot(file['data'][:, pixel_x, pixel_y])
    plt.xlabel('Resultant number', fontsize=16)
    plt.ylabel('Pixel value [DN]', fontsize=16)
    plt.show()

def get_saturated_pixel_count(file_path):
    """Get number of saturated pixels from DQ flags."""
    f = get_asdf_file(file_path)
    # L1 files do not have data quality flags
    if f['romanisim']['level'] == 1:
        return None
    bit = 1  # Saturation bit
    definition = str(dqflags(bit)).split('.')[1]
    n_pix = np.sum(np.bool_(f['dq'].flatten() & (1 << bit)))
    print(f'Bit value {bit} corresponds to {definition}')
    print(f'Number of {definition} pixels: {n_pix:,} ({n_pix / f["dq"].size:.2%})')
    return n_pix

def get_exposure_metadata(file_path):
    """Extract exposure metadata."""
    f = get_roman_dm_file(file_path)
    return f.meta.exposure

def get_exposure_start_time(file_path):
    """Get exposure start time."""
    f = get_roman_dm_file(file_path)
    start_time = Time(f.meta.exposure.start_time, format='isot', scale='utc')
    return start_time

def get_exposure_end_time(file_path):
    """Get exposure end time."""
    f = get_roman_dm_file(file_path)
    end_time = Time(f.meta.exposure.end_time, format='isot', scale='utc')
    return end_time

def get_effective_exposure_time(file_path):
    """Get effective exposure time."""
    f = get_roman_dm_file(file_path)
    effective_exposure_time = f.meta.exposure.effective_exposure_time * u.s
    return effective_exposure_time

def get_exposure_time(file_path):
    """Get exposure time."""
    f = get_roman_dm_file(file_path)
    exposure_time = f.meta.exposure.exposure_time * u.s
    return exposure_time

def get_frame_time(file_path):
    """Get frame time."""
    f = get_roman_dm_file(file_path)
    frame_time = f.meta.exposure.frame_time * u.s
    return frame_time

def get_number_of_resultants(file_path):
    """Get number of resultants."""
    f = get_roman_dm_file(file_path)
    return f.meta.exposure.nresultants

def get_dq_saturated_pixels(file_path):
    """Get DQ saturated pixels."""
    f = get_roman_dm_file(file_path)
    return f.pixels

def get_catalog(file_name):
    """Load catalog from file."""
    catalog = Table.read(file_name, format="ascii.ecsv")
    return catalog

def galactic_to_equatorial(l, b):
    """
    Convert galactic coordinates to equatorial (RA, Dec).
    
    Parameters:
    -----------
    l : float or array
        Galactic longitude in degrees
    b : float or array
        Galactic latitude in degrees
    
    Returns:
    --------
    ra, dec : float or array
        Right Ascension and Declination in degrees
    """
    l = np.asarray(l, dtype=float)
    b = np.asarray(b, dtype=float)
    
    valid_mask = np.isfinite(l) & np.isfinite(b)
    
    if not np.any(valid_mask):
        raise ValueError("No valid galactic coordinates found")
    
    coord = SkyCoord(l * u.degree, b * u.degree, frame='galactic')
    equatorial = coord.icrs
    
    return equatorial.ra.degree, equatorial.dec.degree




def calculate_single_pixel_variance(analyzer, pixel_y, pixel_x):
    """
    Calculate variance of a single pixel across all resultants.
    
    Parameters:
    -----------
    analyzer : RomanL1ResultantAnalyzer
        The analyzer object containing the ramp data
    pixel_y : int
        Y coordinate of pixel
    pixel_x : int
        X coordinate of pixel
    
    Returns:
    --------
    variance : float
        Variance of the pixel values across all resultants
    pixel_data : dict
        Dictionary containing pixel values for each resultant
    """
    
    pixel_data = {}
    values = []
    
    # Extract value for each resultant
    for i in range(analyzer.ramp_data.shape[0]):
        value = analyzer.ramp_data[i, pixel_y, pixel_x]
        pixel_data[f'resultant_{i}'] = value
        if np.isfinite(value):  # Exclude NaN/inf values
            values.append(value)
    
    # Calculate variance
    variance = np.var(values) if len(values) > 0 else np.nan
    
    return variance, pixel_data


def plot_single_pixel_with_variance(analyzer, pixel_y, pixel_x, filename=None):
    """Plot DN value vs resultant for a single pixel and show variance."""
    
    variance, pixel_data = calculate_single_pixel_variance(analyzer, pixel_y, pixel_x)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    resultants = range(len(pixel_data))
    values_list = [pixel_data[f'resultant_{i}'] for i in resultants]
    
    # Plot 1: DN vs Resultant number
    ax1.plot(resultants, values_list, 'o-', markersize=8)
    ax1.set_xlabel('Resultant Number')
    ax1.set_ylabel('DN Value')
    ax1.set_title(f'Pixel [{pixel_y}, {pixel_x}] - DN vs Resultant\nVariance = {variance:.2f} DN²')
    ax1.grid(True, alpha=0.3)
    
    # Add mean line
    mean_val = np.nanmean(values_list)
    ax1.axhline(mean_val, color='red', linestyle='--', alpha=0.5, label=f'Mean = {mean_val:.2f}')
    ax1.legend()
    
    # Plot 2: DN vs Time
    if analyzer.resultant_times is not None:
        ax2.plot(analyzer.resultant_times, values_list, 'o-', markersize=8)
        ax2.set_xlabel('Time (s)')
        ax2.set_ylabel('DN Value')
        ax2.set_title(f'Pixel [{pixel_y}, {pixel_x}] - DN vs Time\nVariance = {variance:.2f} DN²')
        ax2.grid(True, alpha=0.3)
        ax2.axhline(mean_val, color='red', linestyle='--', alpha=0.5, label=f'Mean = {mean_val:.2f}')
        ax2.legend()
    
    # Add filename to suptitle if provided
    if filename:
        fig.suptitle(filename, y=1.02)
    
    plt.tight_layout()
    plt.show()
    
    return variance, pixel_data


def plot_multiple_pixels_variance(analyzer, pixel_coords, filename=None):
    """
    Plot DN values for multiple pixels and compare their variances.
    
    Parameters:
    -----------
    analyzer : RomanL1ResultantAnalyzer
    pixel_coords : list of tuples
        List of (y, x) coordinates to plot
    filename : str, optional
    """
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    variances = {}
    
    for pixel_y, pixel_x in pixel_coords:
        variance, pixel_data = calculate_single_pixel_variance(analyzer, pixel_y, pixel_x)
        resultants = range(len(pixel_data))
        values_list = [pixel_data[f'resultant_{i}'] for i in resultants]
        
        label = f'[{pixel_y}, {pixel_x}] (var={variance:.2f})'
        variances[(pixel_y, pixel_x)] = variance
        
        # Plot 1: DN vs Resultant
        ax1.plot(resultants, values_list, 'o-', label=label, markersize=6, alpha=0.7)
        
        # Plot 2: DN vs Time
        if analyzer.resultant_times is not None:
            ax2.plot(analyzer.resultant_times, values_list, 'o-', label=label, markersize=6, alpha=0.7)
    
    ax1.set_xlabel('Resultant Number')
    ax1.set_ylabel('DN Value')
    ax1.set_title('Multiple Pixels - DN vs Resultant')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    if analyzer.resultant_times is not None:
        ax2.set_xlabel('Time (s)')
        ax2.set_ylabel('DN Value')
        ax2.set_title('Multiple Pixels - DN vs Time')
        ax2.legend()
        ax2.grid(True, alpha=0.3)
    
    if filename:
        fig.suptitle(filename, y=1.02)
    
    plt.tight_layout()
    plt.show()
    
    print("\nPixel Variances:")
    for coords, var in variances.items():
        print(f"  Pixel {coords}: {var:.2f} DN²")
    
    return variances



def detect_source_position(data, search_radius=500, central_region=True):
    """
    Automatically detect the brightest source in the image.
    
    Parameters:
    -----------
    data : np.ndarray
        2D image data
    search_radius : int
        Radius around center to search (if central_region=True)
    central_region : bool
        If True, only search central region (faster, assumes source near center)
    
    Returns:
    --------
    y, x : int, int
        Source position (row, column)
    peak_value : float
        Peak pixel value
    """
    
    if central_region:
        # Search only central region (faster)
        center_y, center_x = data.shape[0] // 2, data.shape[1] // 2
        y_start = max(0, center_y - search_radius)
        y_end = min(data.shape[0], center_y + search_radius)
        x_start = max(0, center_x - search_radius)
        x_end = min(data.shape[1], center_x + search_radius)
        
        search_region = data[y_start:y_end, x_start:x_end]
        
        # Find max in search region
        max_val = np.nanmax(search_region)
        local_max = np.where(search_region == max_val)
        
        # Convert back to full image coordinates
        source_y = y_start + local_max[0][0]
        source_x = x_start + local_max[1][0]
    else:
        # Search entire image
        max_val = np.nanmax(data)
        max_loc = np.where(data == max_val)
        source_y = max_loc[0][0]
        source_x = max_loc[1][0]
    
    return source_y, source_x, max_val

# ======================================================================
# PIPELINE HELPER FUNCTIONS
# ======================================================================

def save_checkpoint(file_number, total_processed):
    """Save checkpoint to resume later if interrupted."""
    checkpoint = {
        'last_completed': file_number,
        'total_processed': total_processed,
        'timestamp': time.time(),
    }
    with open(CHECKPOINT_FILE, 'w') as f:
        json.dump(checkpoint, f)

def load_checkpoint():
    """Load checkpoint to resume processing."""
    try:
        with open(CHECKPOINT_FILE, 'r') as f:
            checkpoint = json.load(f)
        return checkpoint
    except FileNotFoundError:
        return {'last_completed': 0, 'total_processed': 0}

def cleanup_files(*filepaths):
    """Delete files to save space."""
    deleted = []
    for filepath in filepaths:
        if os.path.exists(filepath):
            os.remove(filepath)
            deleted.append(os.path.basename(filepath))
    return deleted

def estimate_completion(file_num, start_time, total_files):
    """Estimate remaining time."""
    elapsed = time.time() - start_time
    if file_num > 0:
        rate = file_num / elapsed
        remaining_files = total_files - file_num
        remaining_seconds = remaining_files / rate
        
        return {
            'elapsed_hours': elapsed / 3600,
            'remaining_hours': remaining_seconds / 3600,
            'remaining_days': remaining_seconds / 86400,
            'percent_complete': (file_num / total_files) * 100
        }
    return None


def cleanup_all_pipeline_files(base_filename, temp_dir):
    """
    Delete ALL .asdf files in temp directory.
    More aggressive cleanup to ensure nothing is left behind.
    """
    deleted = []
    
    # Get ALL .asdf files in the temp directory
    all_asdf_files = glob.glob(os.path.join(temp_dir, '*.asdf'))
    
    for filepath in all_asdf_files:
        try:
            if os.path.exists(filepath):
                os.remove(filepath)
                deleted.append(os.path.basename(filepath))
        except Exception as e:
            print(f"     Could not delete {os.path.basename(filepath)}: {e}")
    
    return deleted



def get_saturated_pixels_and_neighbors_from_dq(dq_array):
    """
    Get saturated pixels and their 8 neighbors from DQ flags.
    
    Masks saturated pixels plus their immediate 8 neighbors (3×3 box).
    This prevents contamination from charge bleeding and crosstalk.
    
    Parameters:
    -----------
    dq_array : 2D array
        DQ flags array from L2 file
    
    Returns:
    --------
    mask : 2D boolean array
        True for saturated pixels and their neighbors
    n_saturated_core : int
        Number of core saturated pixels (not including neighbors)
    n_saturated_total : int
        Total pixels affected (including neighbors)
    """
    from scipy import ndimage
    
    # Get saturated pixels (bit 1 = SATURATED)
    saturated_bit = 1
    saturated_mask = (dq_array & (1 << saturated_bit)) > 0
    
    n_saturated_core = np.sum(saturated_mask)
    
    # Expand to include 8 neighbors (3x3 box around each saturated pixel)
    structure = np.ones((3, 3), dtype=bool)
    expanded_mask = ndimage.binary_dilation(saturated_mask, structure=structure)
    
    n_saturated_total = np.sum(expanded_mask)
    
    return expanded_mask, n_saturated_core, n_saturated_total


def extract_jump_metrics(jump_file, image_id):
    """Extract cosmic ray/jump detection statistics."""
    try:
        with asdf.open(jump_file) as af:
            if 'roman' in af and 'groupdq' in af['roman']:
                groupdq = af['roman']['groupdq']
                
                # Jump flag is bit 2 in groupdq
                jump_flag = 4  # 2^2
                jump_mask = (groupdq & jump_flag) > 0
                
                # Count jumps
                n_jumps_total = np.sum(jump_mask)
                n_pixels_with_jumps = np.sum(np.any(jump_mask, axis=0))
                total_pixels = groupdq.shape[1] * groupdq.shape[2]
                jump_fraction = n_pixels_with_jumps / total_pixels
                
                # Get jump amplitudes if data available
                jump_amplitudes = []
                if 'data' in af['roman']:
                    data = af['roman']['data']
                    for i in range(data.shape[0] - 1):
                        diff = data[i+1] - data[i]
                        jumps = diff[jump_mask[i]]
                        if len(jumps) > 0:
                            jump_amplitudes.extend(jumps)
                
                result = {
                    'image_id': image_id,
                    'n_jumps_total': int(n_jumps_total),
                    'n_pixels_with_jumps': int(n_pixels_with_jumps),
                    'jump_fraction': float(jump_fraction),
                    'mean_jump_amplitude': float(np.mean(jump_amplitudes)) if jump_amplitudes else 0.0,
                    'max_jump_amplitude': float(np.max(jump_amplitudes)) if jump_amplitudes else 0.0,
                    'median_jump_amplitude': float(np.median(jump_amplitudes)) if jump_amplitudes else 0.0,
                    'jumps_per_pixel_mean': float(n_jumps_total / total_pixels),
                }
                
                return result
    except Exception as e:
        pass
    
    return None


def extract_wcs_for_sources(wcs_file, image_id, source_positions):
    """Extract RA/Dec coordinates for source positions."""
    results = []
    try:
        with asdf.open(wcs_file) as af:
            if 'roman' in af and hasattr(af['roman'], 'meta'):
                wcs_info = af['roman'].meta.wcs
                
                for i, (x, y) in enumerate(source_positions):
                    try:
                        # Convert pixel to world coordinates
                        ra, dec = wcs_info.pixel_to_world_values(x, y)
                        
                        # Estimate uncertainty (typically ~0.1 arcsec for Roman)
                        ra_uncertainty = 0.1 / 3600  # degrees
                        dec_uncertainty = 0.1 / 3600
                        
                        results.append({
                            'image_id': image_id,
                            'source_id': i + 1,
                            'x_pixel': float(x),
                            'y_pixel': float(y),
                            'ra_deg': float(ra),
                            'dec_deg': float(dec),
                            'ra_uncertainty': float(ra_uncertainty),
                            'dec_uncertainty': float(dec_uncertainty),
                        })
                    except:
                        pass
    except Exception as e:
        pass
    
    return results


def extract_ramp_fit_quality(rampfit_file, image_id):
    """Extract ramp fit quality metrics."""
    try:
        with asdf.open(rampfit_file) as af:
            if 'roman' in af:
                data = af['roman']
                
                result = {
                    'image_id': image_id,
                    'mean_chi_squared': 0.0,
                    'median_chi_squared': 0.0,
                    'var_poisson_mean': 0.0,
                    'var_rnoise_mean': 0.0,
                    'n_groups_mean': 0.0,
                    'good_fit_fraction': 0.0,
                }
                
                if 'var_poisson' in data:
                    result['var_poisson_mean'] = float(np.nanmean(data['var_poisson']))
                
                if 'var_rnoise' in data:
                    result['var_rnoise_mean'] = float(np.nanmean(data['var_rnoise']))
                
                if 'chi_squared' in data:
                    chi_sq = data['chi_squared']
                    result['mean_chi_squared'] = float(np.nanmean(chi_sq))
                    result['median_chi_squared'] = float(np.nanmedian(chi_sq))
                    result['good_fit_fraction'] = float(np.sum(chi_sq < 2.0) / chi_sq.size)
                
                return result
    except Exception as e:
        pass
    
    return None


def extract_dq_flags_summary(l2_file, image_id):
    """Extract DQ flag statistics from L2 file."""
    try:
        with asdf.open(l2_file) as af:
            if 'roman' in af and 'dq' in af['roman']:
                dq = af['roman']['dq']
                total_pixels = dq.size
                
                # DQ bit definitions
                flags = {
                    'do_not_use': 0, 'saturated': 1, 'jump_det': 2, 'dropout': 3,
                    'outlier': 4, 'persistence': 5, 'ad_floor': 6, 
                    'other_bad_pixel': 7, 'reference_pixel': 8,
                    'hot': 9, 'warm': 10, 'dead': 11, 'telegraph': 12,
                }
                
                result = {'image_id': image_id}
                
                for flag_name, bit in flags.items():
                    count = np.sum((dq & (1 << bit)) > 0)
                    result[flag_name] = float(count / total_pixels * 100)
                
                return result
    except Exception as e:
        pass
    
    return None


print("Helper functions loaded")



# ======================================================================
# UNCERTAINTY ANALYSIS FUNCTIONS (Post-processing)
# ======================================================================

def plot_uncertainty_by_steps(csv_filename, mag, plot_style='side_by_side', output_dir='./analysis_results'):
    """
    Plot photometry uncertainties as a function of image steps.
    
    Parameters:
    -----------
    csv_filename : str
        Path to the combined photometry CSV file
    mag : int
        Magnitude value for titles and labels
    plot_style : str
        'side_by_side': Two plots side by side
        'normalized': Both on same plot, normalized 0-1
        'dual_axis': Clean dual y-axis plot
        'log_ratio': Plot the ratio of uncertainties
    output_dir : str
        Directory to save plots
    """
    from scipy import stats
    
    print(f"Loading photometry data from {csv_filename}...")
    
    try:
        df = pd.read_csv(csv_filename)
        print(f"  Loaded {len(df)} measurements from {len(df['image_id'].unique())} images")
    except FileNotFoundError:
        print(f"  Error: Could not find file '{csv_filename}'")
        return
    except Exception as e:
        print(f"  Error loading CSV: {e}")
        return
    
    # Calculate summary statistics per image
    summary_stats = []
    
    for image_id in sorted(df['image_id'].unique()):
        img_data = df[df['image_id'] == image_id]
        
        flux_err = img_data['flux_jy_err'].dropna()
        mag_err = img_data['magnitude_err'].dropna()
        snr_data = img_data['snr'].dropna()
        
        stats_dict = {
            'image_id': image_id,
            'n_stars': len(img_data),
            'flux_err_mean': flux_err.mean() if len(flux_err) > 0 else np.nan,
            'flux_err_std': flux_err.std() if len(flux_err) > 0 else np.nan,
            'flux_err_median': flux_err.median() if len(flux_err) > 0 else np.nan,
            'mag_err_mean': mag_err.mean() if len(mag_err) > 0 else np.nan,
            'mag_err_std': mag_err.std() if len(mag_err) > 0 else np.nan,
            'mag_err_median': mag_err.median() if len(mag_err) > 0 else np.nan,
            'snr_mean': snr_data.mean() if len(snr_data) > 0 else np.nan,
            'snr_std': snr_data.std() if len(snr_data) > 0 else np.nan,
            'snr_median': snr_data.median() if len(snr_data) > 0 else np.nan
        }
        summary_stats.append(stats_dict)
    
    summary_df = pd.DataFrame(summary_stats)
    print(f"\nSummary statistics by image:")
    print(summary_df.round(6))
    
    # Create plot based on style
    if plot_style == 'side_by_side':
        _create_side_by_side_plot(summary_df, df, mag, output_dir)
    elif plot_style == 'dual_axis':
        _create_dual_axis_plot(summary_df, df, mag, output_dir)
    else:
        print(f"Plot style '{plot_style}' not implemented. Using 'side_by_side'.")
        _create_side_by_side_plot(summary_df, df, mag, output_dir)
    
    # Trend analysis
    _print_trend_analysis(summary_df)
    
    # Save summary statistics
    summary_filename = os.path.join(output_dir, f'Uncertainty_Summary_by_Steps_Mag_{mag}.csv')
    summary_df.to_csv(summary_filename, index=False)
    print(f"\nSummary statistics saved as: {summary_filename}")
    
    return summary_df


def _create_side_by_side_plot(summary_df, df, mag, output_dir):
    """Create side-by-side subplots."""
    from scipy import stats
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Left plot: Flux Uncertainty
    ax1.errorbar(summary_df['image_id'], summary_df['flux_err_mean'], 
                yerr=summary_df['flux_err_std'], fmt='bo-', markersize=10, 
                linewidth=3, capsize=8, label='Mean ± Std', color='blue')
    ax1.plot(summary_df['image_id'], summary_df['flux_err_median'], 'bs--', 
             markersize=8, linewidth=2, alpha=0.8, color='lightblue', label='Median')
    
    ax1.set_xlabel('Image Step', fontsize=14, fontweight='bold')
    ax1.set_ylabel('Flux Uncertainty (Jy)', fontsize=14, fontweight='bold')
    ax1.set_title(f'Flux Uncertainty vs Image Steps (Mag {mag})', 
                  fontsize=14, fontweight='bold')
    ax1.set_yscale('log')
    ax1.grid(True, alpha=0.3)
    ax1.legend(fontsize=10)
    
    # Right plot: Magnitude Uncertainty
    ax2.errorbar(summary_df['image_id'], summary_df['mag_err_mean'], 
                yerr=summary_df['mag_err_std'], fmt='ro-', markersize=10, 
                linewidth=3, capsize=8, label='Mean ± Std', color='red')
    ax2.plot(summary_df['image_id'], summary_df['mag_err_median'], 'rs--', 
             markersize=8, linewidth=2, alpha=0.8, color='lightcoral', label='Median')
    
    ax2.set_xlabel('Image Step', fontsize=14, fontweight='bold')
    ax2.set_ylabel('Magnitude Uncertainty (mag)', fontsize=14, fontweight='bold')
    ax2.set_title(f'Magnitude Uncertainty vs Image Steps (Mag {mag})', 
                  fontsize=14, fontweight='bold')
    ax2.set_yscale('log')
    ax2.grid(True, alpha=0.3)
    ax2.legend(fontsize=10)
    
    # Add trend annotations
    if len(summary_df) > 1:
        y_flux = summary_df['flux_err_mean'].dropna().values
        x_flux = summary_df.loc[summary_df['flux_err_mean'].notna(), 'image_id'].values
        if len(x_flux) > 1:
            slope_flux, _, r_flux, _, _ = stats.linregress(x_flux, y_flux)
            trend_flux = "↗ increasing" if slope_flux > 0 else "↘ decreasing"
            ax1.text(0.02, 0.95, f'Trend: {trend_flux}\n(r={r_flux:.3f})', 
                    transform=ax1.transAxes, fontsize=10, verticalalignment='top',
                    bbox=dict(boxstyle='round,pad=0.3', facecolor='lightblue', alpha=0.8))
        
        y_mag = summary_df['mag_err_mean'].dropna().values
        x_mag = summary_df.loc[summary_df['mag_err_mean'].notna(), 'image_id'].values
        if len(x_mag) > 1:
            slope_mag, _, r_mag, _, _ = stats.linregress(x_mag, y_mag)
            trend_mag = "↗ increasing" if slope_mag > 0 else "↘ decreasing"
            ax2.text(0.02, 0.95, f'Trend: {trend_mag}\n(r={r_mag:.3f})', 
                    transform=ax2.transAxes, fontsize=10, verticalalignment='top',
                    bbox=dict(boxstyle='round,pad=0.3', facecolor='lightcoral', alpha=0.8))
    
    plt.tight_layout()
    plot_filename = os.path.join(output_dir, f'Uncertainty_SideBySide_Mag_{mag}.png')
    plt.savefig(plot_filename, dpi=300, bbox_inches='tight')
    print(f"\nSide-by-side uncertainty plot saved as: {plot_filename}")
    plt.close()


def _create_dual_axis_plot(summary_df, df, mag, output_dir):
    """Create clean dual y-axis plot."""
    fig, ax1 = plt.subplots(figsize=(12, 7))
    ax2 = ax1.twinx()
    
    # Plot flux uncertainty (blue)
    ax1.errorbar(summary_df['image_id'], summary_df['flux_err_mean'], 
                yerr=summary_df['flux_err_std'], fmt='bo-', markersize=10, 
                linewidth=3, capsize=8, color='blue', alpha=0.8)
    
    # Plot magnitude uncertainty (red)
    ax2.errorbar(summary_df['image_id'], summary_df['mag_err_mean'], 
                 yerr=summary_df['mag_err_std'], fmt='ro-', markersize=10, 
                 linewidth=3, capsize=8, color='red', alpha=0.8)
    
    ax1.set_xlabel('Image Step', fontsize=14, fontweight='bold')
    ax1.set_ylabel('Flux Uncertainty (Jy)', color='blue', fontsize=14, fontweight='bold')
    ax2.set_ylabel('Magnitude Uncertainty (mag)', color='red', fontsize=14, fontweight='bold')
    ax1.set_title(f'Flux & Magnitude Uncertainties vs Image Steps (Mag {mag})', 
                  fontsize=16, fontweight='bold')
    
    ax1.set_yscale('log')
    ax2.set_yscale('log')
    ax1.grid(True, alpha=0.3)
    ax1.tick_params(axis='y', labelcolor='blue', labelsize=12)
    ax2.tick_params(axis='y', labelcolor='red', labelsize=12)
    
    plt.tight_layout()
    plot_filename = os.path.join(output_dir, f'Uncertainty_DualAxis_Mag_{mag}.png')
    plt.savefig(plot_filename, dpi=300, bbox_inches='tight')
    print(f"\nDual-axis uncertainty plot saved as: {plot_filename}")
    plt.close()


def _print_trend_analysis(summary_df):
    """Print detailed trend analysis."""
    from scipy import stats
    
    print(f"\nTREND ANALYSIS:")
    print("=" * 50)
    
    # Flux uncertainty trend
    y_flux = summary_df['flux_err_mean'].dropna().values
    x_flux = summary_df.loc[summary_df['flux_err_mean'].notna(), 'image_id'].values
    if len(x_flux) > 1:
        slope_flux, _, r_flux, p_flux, _ = stats.linregress(x_flux, y_flux)
        print(f"Flux uncertainty trend:")
        print(f"   Slope: {slope_flux:.2e} Jy/step")
        print(f"   R-correlation: {r_flux:.3f}")
        print(f"   P-value: {p_flux:.3f}")
        trend_flux = "increasing" if slope_flux > 0 else "decreasing"
        print(f"   Overall trend: {trend_flux}")
    
    # Magnitude uncertainty trend
    y_mag = summary_df['mag_err_mean'].dropna().values
    x_mag = summary_df.loc[summary_df['mag_err_mean'].notna(), 'image_id'].values
    if len(x_mag) > 1:
        slope_mag, _, r_mag, p_mag, _ = stats.linregress(x_mag, y_mag)
        print(f"\nMagnitude uncertainty trend:")
        print(f"   Slope: {slope_mag:.4f} mag/step")
        print(f"   R-correlation: {r_mag:.3f}")
        print(f"   P-value: {p_mag:.3f}")
        trend_mag = "increasing" if slope_mag > 0 else "decreasing"
        print(f"   Overall trend: {trend_mag}")


# ============================================================================
# SETUP
# ============================================================================

print(f"\n{'='*80}")
print(f"SPACE-EFFICIENT PIPELINE - FULLY SELF-CONTAINED")
print(f"{'='*80}")
print(f"Observation period: {OBSERVATION_PERIOD_DAYS} days")
print(f"Cadence: {CADENCE_MINUTES} minutes ({CADENCE_SECONDS} seconds)")
print(f"Total exposures: {N_EXPOSURES}")
print(f"Estimated processing time: ~{N_EXPOSURES * 210 / 86400:.1f} days")
print(f"{'='*80}")

# Create directories
os.makedirs(RESULTS_DIR, exist_ok=True)
os.makedirs(TEMP_L1_DIR, exist_ok=True)

# Initialize CSV files
if not os.path.exists(L1_METRICS_CSV):
    l1_columns = ['file_number', 'timestamp_sec', 'timestamp_hours', 'timestamp_days',
                  'exposure_time', 'n_resultants', 'mean_slope', 'median_slope', 
                  'std_slope', 'min_slope', 'max_slope', 'n_saturated_pixels', 
                  'saturation_fraction', 'n_saturated_with_neighbors', 
                  'saturation_fraction_with_neighbors', 'n_valid_pixels',
                  'ma_table_number', 'frame_time', 'read_noise_used', 
                  'detector', 'optical_element']
    pd.DataFrame(columns=l1_columns).to_csv(L1_METRICS_CSV, index=False)
    print(f" Initialized {L1_METRICS_CSV}")

if not os.path.exists(PHOTOMETRY_CSV):
    photometry_columns = ['image_id', 'id', 'xcentroid', 'ycentroid', 'sum', 'sum_err',
                         'flux_jy', 'flux_jy_final', 'flux_jy_err', 
                         'magnitude', 'magnitude_err', 'snr']
    pd.DataFrame(columns=photometry_columns).to_csv(PHOTOMETRY_CSV, index=False)
    print(f" Initialized {PHOTOMETRY_CSV}")

if not os.path.exists(PERSISTENCE_CSV):
    persistence_columns = ['file_number', 'timestamp_sec', 'timestamp_hours', 'timestamp_days',
                          'n_persistence_pixels', 'persistence_fraction', 
                          'persistence_strength', 'persistence_max', 'diff_mean', 'diff_std']
    pd.DataFrame(columns=persistence_columns).to_csv(PERSISTENCE_CSV, index=False)
    print(f" Initialized {PERSISTENCE_CSV}")


# Additional CSV files for intermediate data
JUMP_METRICS_CSV = os.path.join(RESULTS_DIR, 'jump_detection_metrics.csv')
WCS_METRICS_CSV = os.path.join(RESULTS_DIR, 'wcs_astrometry.csv')
RAMP_QUALITY_CSV = os.path.join(RESULTS_DIR, 'ramp_fit_quality.csv')
DQ_FLAGS_CSV = os.path.join(RESULTS_DIR, 'dq_flags_summary.csv')


# Initialize intermediate data CSVs
if EXTRACT_JUMPS and not os.path.exists(JUMP_METRICS_CSV):
    jump_columns = ['image_id', 'n_jumps_total', 'n_pixels_with_jumps', 
                   'jump_fraction', 'mean_jump_amplitude', 'max_jump_amplitude',
                   'median_jump_amplitude', 'jumps_per_pixel_mean']
    pd.DataFrame(columns=jump_columns).to_csv(JUMP_METRICS_CSV, index=False)
    print(f" Initialized {JUMP_METRICS_CSV}")

if EXTRACT_WCS and not os.path.exists(WCS_METRICS_CSV):
    wcs_columns = ['image_id', 'source_id', 'x_pixel', 'y_pixel', 
                  'ra_deg', 'dec_deg', 'ra_uncertainty', 'dec_uncertainty']
    pd.DataFrame(columns=wcs_columns).to_csv(WCS_METRICS_CSV, index=False)
    print(f" Initialized {WCS_METRICS_CSV}")

if EXTRACT_RAMP_QUALITY and not os.path.exists(RAMP_QUALITY_CSV):
    quality_columns = ['image_id', 'mean_chi_squared', 'median_chi_squared',
                      'var_poisson_mean', 'var_rnoise_mean', 'n_groups_mean',
                      'good_fit_fraction']
    pd.DataFrame(columns=quality_columns).to_csv(RAMP_QUALITY_CSV, index=False)
    print(f" Initialized {RAMP_QUALITY_CSV}")

if EXTRACT_DQ_FLAGS and not os.path.exists(DQ_FLAGS_CSV):
    dq_columns = ['image_id', 'do_not_use', 'saturated', 'jump_det', 'dropout',
                 'outlier', 'persistence', 'ad_floor', 'other_bad_pixel',
                 'reference_pixel', 'hot', 'warm', 'dead', 'telegraph']
    pd.DataFrame(columns=dq_columns).to_csv(DQ_FLAGS_CSV, index=False)
    print(f" Initialized {DQ_FLAGS_CSV}")


# Check catalog exists
catalog_file = f'./{CATALOG_FILE_NAME}'
if not os.path.exists(catalog_file):
    print(f"\n✗ ERROR: Catalog file not found: {catalog_file}")
    print(f"  Please create the star catalog first!")
    raise FileNotFoundError(f"Catalog file {catalog_file} not found")

print(f" Using catalog: {catalog_file}")
print(f" Photometry will use auto-detected source positions")

# ============================================================================
# MAIN PROCESSING LOOP
# ============================================================================

checkpoint = load_checkpoint()
start_file = checkpoint['last_completed'] + 1
total_processed = checkpoint['total_processed']

if start_file > 1:
    print(f"\n{'='*80}")
    print(f"RESUMING from file {start_file}")
    print(f"Previously processed: {total_processed} files")
    print(f"{'='*80}\n")
else:
    print(f"\n{'='*80}")
    print(f"STARTING PROCESSING")
    print(f"Total files: {N_EXPOSURES}")
    print(f"{'='*80}\n")

reference_image = None

# If resuming, load reference image from file 1's L2
if start_file > 1:
    ref_l2_file = os.path.join(TEMP_L1_DIR, 'roman_sim_F146_WFI01_L2_1.asdf')
    if os.path.exists(ref_l2_file):
        with asdf.open(ref_l2_file) as af:
            reference_image = af['roman']['data'][:]
        print(f" Loaded reference image from file 1 for persistence comparison")
    else:
        print(f"️ WARNING: Reference image not found! Persistence detection disabled.")
        print(f"   Expected: {ref_l2_file}")

processing_start_time = time.time()

for file_num in range(start_file, N_EXPOSURES + 1):
    
    timestamp_sec = (file_num - 1) * CADENCE_SECONDS
    timestamp_hours = timestamp_sec / 3600
    timestamp_days = timestamp_sec / 86400
    
    base_time_dt = datetime.fromisoformat(BASE_TIME)
    obs_time = base_time_dt + timedelta(seconds=timestamp_sec)
    obs_time_iso = obs_time.isoformat()
    
    print(f"\n{'='*80}")
    print(f"Processing {file_num}/{N_EXPOSURES} ({file_num/N_EXPOSURES*100:.2f}%)")
    print(f"Time: Day {timestamp_days:.2f} ({timestamp_hours:.1f} hrs)")
    
    if file_num % 10 == 0 and file_num > start_file:
        est = estimate_completion(file_num - start_file + 1, processing_start_time, N_EXPOSURES - start_file + 1)
        if est:
            print(f"Progress: {est['percent_complete']:.1f}% | Remaining: {est['remaining_days']:.1f} days | Last checkpoint: {file_num}")
    
    print(f"{'='*80}")
    
    try:
        # STEP 1: Generate L1 with persistence modeling
        print("Step 1/6: Generating L1...")
        
        # Use previous L1 file for persistence modeling (if exists)
        prev_file = None
        if file_num > 1:
            # Look for the previous file's L1
            prev_l1_name = f"roman_sim_{FILTER}_WFI{SCA:02d}_L1_{file_num-1}.asdf"
            prev_l1_path = os.path.join(TEMP_L1_DIR, prev_l1_name)
            
            # Check if previous file exists
            if os.path.exists(prev_l1_path):
                prev_file = prev_l1_path
                print(f"  Using previous file for persistence: {os.path.basename(prev_file)}")
            else:
                print(f"  No previous file found - starting fresh")
        
        l1_filepath = run_romanisim_simulationapi(
            catalog_file=catalog_file,
            filter_name=FILTER,
            sca=SCA,
            level=1,
            exposure_params={'ma_table_number': MA_TABLE, 'seed': SEED + file_num},
            pointing=tuple(FIELD_CENTER),
            date=obs_time_iso,
            output_directory=TEMP_L1_DIR,
            output_suffix=f"_{file_num}",
            previous_file=prev_file,  # PERSISTENCE MODELING
            usecrds=True
        )
        l1_filename = os.path.basename(l1_filepath)
        print(f"   Generated: {l1_filename}")
        
        # STEP 2: L1 Analysis with Saturation Replacement (Master method)
        print("Step 2/6: Analyzing L1 and replacing saturated pixels...")
        analyzer = RomanL1ResultantAnalyzer(l1_filepath)
        analyzer.load_l1_file()
        
        # Extract resultants
        resultants = analyzer.extract_all_resultants()
        
        # Get saturation threshold (65k DN for Roman WFI)
        saturated_level = 60000  # DN (matches Master notebook threshold)
        replacement_value = np.nan
        
        # MASTER METHOD: Progressive saturation accumulation (YOUR EXACT LOGIC)
        accumulated_saturated_pixels = set()
        modified_resultants = []
        
        print("  Replacing saturated pixels in resultants...")
        
        i = 0
        for resultant_name in sorted(resultants.keys()):
            # Start with a copy of the current resultant - CONVERT TO FLOAT
            data = resultants[resultant_name].astype(np.float64)
            
            # DIAGNOSTIC: Show actual pixel values BEFORE saturation check
            if i == 0:  # Only for first resultant
                print(f"\n  🔍 DIAGNOSTIC (Resultant 0 - Original Data):")
                print(f"    Global stats: min={np.nanmin(data):.1f}, max={np.nanmax(data):.1f}, mean={np.nanmean(data):.1f}")
                print(f"    Pixels > 50000 DN: {np.sum(data > 50000)}")
                print(f"    Pixels > 60000 DN: {np.sum(data > 60000)}")
                print(f"    Pixels >= 65000 DN (saturated): {np.sum(data >= saturated_level)}")
                print(f"    Saturation threshold: {saturated_level} DN")
            
            # First, apply accumulated saturated pixels from PREVIOUS resultants
            # This maintains the replacement value for previously saturated pixels
            if accumulated_saturated_pixels:
                data = replace_saturated_pixels(
                    data, list(accumulated_saturated_pixels), replacement_value
                )
            
            # Now find NEW saturated pixels in the current (modified) resultant
            new_saturated_pixels = get_saturated_pixels_and_neighbors(
                data, saturated_level
            )
            
            # Add new saturated pixels to accumulated set
            accumulated_saturated_pixels.update(new_saturated_pixels)
            
            # Apply the new saturated pixels to current resultant
            data = replace_saturated_pixels(
                data, new_saturated_pixels, replacement_value
            )
            
            print(f"  Resultant {i}: New saturated pixels: {len(new_saturated_pixels)}, "
                  f"Total accumulated: {len(accumulated_saturated_pixels)}")
            
            modified_resultants.append(data)
            i += 1
        
        print(f"   Replaced saturated pixels across {len(modified_resultants)} resultants")
        print(f"\nFile complete. Total saturated pixels across all resultants: "
              f"{len(accumulated_saturated_pixels)}")
        
        # Save cleaned L1 file
        l1_cleaned_filepath = update_unsaturated_resultant(l1_filepath, modified_resultants)
        print(f"   Saved cleaned L1: {os.path.basename(l1_cleaned_filepath)}")
        
        # Now analyze the CLEANED L1
        analyzer_cleaned = RomanL1ResultantAnalyzer(l1_cleaned_filepath)
        analyzer_cleaned.load_l1_file()
        analyzer_cleaned.reference_pixel_correction()
        
        # SPEED OPTIMIZATION: Skip manual Casertano fitting
        # romancal's ramp_fit step does this for us (optimized in C)
        print(f"   Skipped manual Casertano fitting (using romancal's output)")
        
        # Create dummy slope arrays (not used - romancal L2 has the real slopes)
        slopes = np.zeros(analyzer_cleaned.ramp_data.shape[1:])
        slope_errors = np.zeros(analyzer_cleaned.ramp_data.shape[1:])
        valid_slopes = slopes.flatten()
        
        # Saturation detection will be done from L2 DQ flags (more efficient & accurate)
        n_saturated = 0  # Placeholder - will get from L2
        
        l1_metrics = {
            'file_number': file_num,
            'timestamp_sec': timestamp_sec,
            'timestamp_hours': timestamp_hours,
            'timestamp_days': timestamp_days,
            'exposure_time': analyzer_cleaned.exposure_time,
            'n_resultants': len(analyzer_cleaned.resultant_times),
            'mean_slope': np.mean(valid_slopes),
            'median_slope': np.median(valid_slopes),
            'std_slope': np.std(valid_slopes),
            'min_slope': np.min(valid_slopes),
            'max_slope': np.max(valid_slopes),
            'n_saturated_pixels': n_saturated,
            'saturation_fraction': n_saturated / slopes.size,
            'n_saturated_with_neighbors': n_saturated_total if 'n_saturated_total' in locals() else n_saturated,
            'saturation_fraction_with_neighbors': (n_saturated_total if 'n_saturated_total' in locals() else n_saturated) / slopes.size,
            'n_valid_pixels': len(valid_slopes),
        }
        
        pd.DataFrame([l1_metrics]).to_csv(L1_METRICS_CSV, mode='a', header=False, index=False)
        print(f"   Slope: {l1_metrics['mean_slope']:.4f} DN/s, Sat: {l1_metrics['saturation_fraction']*100:.2f}%")
        
        # STEP 3: Pipeline (using CLEANED L1)
        print("Step 3/6: Running pipeline on cleaned L1...")
        pipeline = ExposurePipeline()
        pipeline.save_results = False  # Don't save final results
        
        # Disable ALL intermediate step outputs
        if hasattr(pipeline.steps, 'linearity'):
            pipeline.steps.linearity.save_results = False
        if hasattr(pipeline.steps, 'dark_current'):
            pipeline.steps.dark_current.save_results = False
        if hasattr(pipeline.steps, 'jump'):
            pipeline.steps.jump.save_results = False
        if hasattr(pipeline.steps, 'ramp_fit'):
            pipeline.steps.ramp_fit.save_results = False
        if hasattr(pipeline.steps, 'assign_wcs'):
            pipeline.steps.assign_wcs.save_results = False
        if hasattr(pipeline.steps, 'flat_field'):
            pipeline.steps.flat_field.save_results = False
        if hasattr(pipeline.steps, 'photom'):
            pipeline.steps.photom.save_results = False
        
        # Skip source detection to avoid creating large extra files
        # (we do our own source detection anyway)
        if hasattr(pipeline.steps, 'source_detection'):
            pipeline.steps.source_detection.skip = True
            print(f"   Disabled source_detection step (saves disk space)")
        if hasattr(pipeline.steps, 'source_detection'):
            pipeline.steps.source_detection.save_results = False
        
        l2_model = pipeline.process(l1_cleaned_filepath)  # Use CLEANED L1
        
        l2_filename = l1_filename.replace('L1', 'L2').replace('_1', '_cal_1')
        l2_filepath = os.path.join(TEMP_L1_DIR, l2_filename)
        l2_model.save(l2_filepath)
        print(f"   Generated: {l2_filename}")
        
        # STEP 4: Extract saturation stats and photometry
        print("Step 4/6: Photometry (saturated pixels already removed in L1)...")
        with asdf.open(l2_filepath) as af:
            data = af['roman']['data']
            err = af['roman']['err'] if 'err' in af['roman'] else None
            
            # Get saturation stats from DQ flags (for tracking)
            if 'dq' in af['roman']:
                dq = af['roman']['dq']
                saturated_mask_with_neighbors, n_saturated, n_saturated_total = get_saturated_pixels_and_neighbors_from_dq(dq)
                print(f"  Saturation (from L2 DQ): {n_saturated} core, {n_saturated_total} total")
            else:
                n_saturated = 0
                n_saturated_total = 0
            
            # Data is already clean (saturated pixels replaced in L1)
            data_cleaned = data
            
            if file_num == 1:
                reference_image = data.copy()
                print("   Saved reference image")
            
            # AUTO-DETECT source position in this L2 image
            source_y, source_x, peak_value = detect_source_position(data)
            print(f"  Auto-detected source: ({source_y}, {source_x})")
            print(f"  Peak value: {peak_value:.2f} DN")
            
            # Update source_positions for photometry
            source_positions = [(source_x, source_y)]  # photutils uses (x, y) order
            print(f"   Using detected position for photometry")
            print(f"  DEBUG: source_positions = {source_positions}")
            print(f"  DEBUG: About to run photometry loop for {len(source_positions)} sources...")
            
            for i, (x, y) in enumerate(source_positions):
                print(f"  DEBUG: Processing source {i+1} at ({x}, {y})")
                aperture = CircularAperture((x, y), r=10)
                annulus = CircularAnnulus((x, y), r_in=15, r_out=25)
                
                phot_table = aperture_photometry(data_cleaned, aperture)
                bkg_table = aperture_photometry(data, annulus)
                
                # Background-subtracted flux (in DN)
                bkg_mean = bkg_table['aperture_sum'][0] / annulus.area
                aperture_sum_dn = phot_table['aperture_sum'][0] - bkg_mean * aperture.area
                
                # Error estimate (in DN)
                aperture_sum_err_dn = 0.0
                if err is not None:
                    err_table = aperture_photometry(err**2, aperture)
                    aperture_sum_err_dn = np.sqrt(err_table['aperture_sum'][0])
                
                # Convert DN to Jy (keep for compatibility but we'll use ZP for magnitude)
                flux_jy = aperture_sum_dn * COUNTS_TO_JY
                flux_jy_err = aperture_sum_err_dn * COUNTS_TO_JY
                
                # Calculate magnitude using instrumental zero point
                # This is more robust and standard for HST/Roman photometry
                if aperture_sum_dn > 0:
                    # AB magnitude: m_AB = ZP - 2.5*log10(counts)
                    # For Roman F146, ZP_AB ~= 28.08 (from CRDS)
                    magnitude = ZERO_POINT_AB - 2.5 * np.log10(aperture_sum_dn)
                    
                    # Error propagation: dm = 1.0857 * (err/flux)
                    magnitude_err = 1.0857 * (aperture_sum_err_dn / aperture_sum_dn) if aperture_sum_dn > 0 else np.nan
                else:
                    magnitude = np.nan
                    magnitude_err = np.nan
                
                # Calculate SNR from counts (more accurate)
                snr = aperture_sum_dn / aperture_sum_err_dn if aperture_sum_err_dn > 0 else np.nan
                
                # Centroid
                xcentroid = x if flux_jy > 0 else np.nan
                ycentroid = y if flux_jy > 0 else np.nan
                
                phot_result = {
                    'image_id': file_num,
                    'id': i + 1,
                    'xcentroid': xcentroid,
                    'ycentroid': ycentroid,
                    'sum': aperture_sum_dn,
                    'sum_err': aperture_sum_err_dn,
                    'flux_jy': flux_jy,
                    'flux_jy_final': flux_jy,
                    'flux_jy_err': flux_jy_err,
                    'magnitude': magnitude,
                    'magnitude_err': magnitude_err,
                    'snr': snr,
                }
                
                pd.DataFrame([phot_result]).to_csv(PHOTOMETRY_CSV, mode='a', header=False, index=False)
                print(f"  DEBUG: Saved photometry result to {PHOTOMETRY_CSV}")
        
        print(f"   Measured {len(source_positions)} sources")
        

        # STEP 4.5: Extract intermediate file data (if enabled)
        if EXTRACT_JUMPS or EXTRACT_WCS or EXTRACT_RAMP_QUALITY or EXTRACT_DQ_FLAGS:
            print("Step 4.5/6: Extracting intermediate file metrics...")
            
            # Find intermediate files
            base = os.path.splitext(l1_filename)[0]
            jump_file = os.path.join(TEMP_L1_DIR, f"{base}_jump.asdf")
            wcs_file = os.path.join(TEMP_L1_DIR, f"{base}_assign_wcs.asdf")
            rampfit_file = os.path.join(TEMP_L1_DIR, f"{base}_rampfit.asdf")
            
            # Extract jump detection metrics
            if EXTRACT_JUMPS and os.path.exists(jump_file):
                jump_metrics = extract_jump_metrics(jump_file, file_num)
                if jump_metrics:
                    pd.DataFrame([jump_metrics]).to_csv(JUMP_METRICS_CSV, mode='a', header=False, index=False)
                    print(f"   Jumps: {jump_metrics['n_jumps_total']} cosmic rays")
            
            # Extract WCS for sources
            if EXTRACT_WCS and os.path.exists(wcs_file):
                wcs_results = extract_wcs_for_sources(wcs_file, file_num, source_positions)
                if wcs_results:
                    pd.DataFrame(wcs_results).to_csv(WCS_METRICS_CSV, mode='a', header=False, index=False)
                    print(f"   WCS: RA/Dec for {len(wcs_results)} sources")
            
            # Extract ramp fit quality
            if EXTRACT_RAMP_QUALITY and os.path.exists(rampfit_file):
                quality_metrics = extract_ramp_fit_quality(rampfit_file, file_num)
                if quality_metrics:
                    pd.DataFrame([quality_metrics]).to_csv(RAMP_QUALITY_CSV, mode='a', header=False, index=False)
                    print(f"   Ramp quality: X² = {quality_metrics['mean_chi_squared']:.3f}")
            
            # Extract DQ flags from L2
            if EXTRACT_DQ_FLAGS:
                dq_metrics = extract_dq_flags_summary(l2_filepath, file_num)
                if dq_metrics:
                    pd.DataFrame([dq_metrics]).to_csv(DQ_FLAGS_CSV, mode='a', header=False, index=False)
                    print(f"   DQ flags: {dq_metrics['saturated']:.2f}% saturated")
        
        # STEP 5: Persistence
        print(f"  DEBUG: reference_image is {'set' if reference_image is not None else 'None'}")
        if reference_image is not None:
            print("Step 5/6: Detecting persistence...")
            with asdf.open(l2_filepath) as af:
                current_data = af['roman']['data']
                diff = current_data - reference_image
                diff_std = np.nanstd(diff)
                diff_mean = np.nanmean(diff)
                
                persistence_mask = np.abs(diff - diff_mean) > (5.0 * diff_std)
                n_pers = np.sum(persistence_mask)
                
                pers_result = {
                    'file_number': file_num,
                    'timestamp_sec': timestamp_sec,
                    'timestamp_hours': timestamp_hours,
                    'timestamp_days': timestamp_days,
                    'n_persistence_pixels': n_pers,
                    'persistence_fraction': n_pers / diff.size,
                    'persistence_strength': np.mean(diff[persistence_mask]) if n_pers > 0 else 0.0,
                    'persistence_max': np.max(np.abs(diff[persistence_mask])) if n_pers > 0 else 0.0,
                    'diff_mean': diff_mean,
                    'diff_std': diff_std,
                }
                
                pd.DataFrame([pers_result]).to_csv(PERSISTENCE_CSV, mode='a', header=False, index=False)
                print(f"   Persistence: {pers_result['persistence_fraction']*100:.4f}%")
        
        # STEP 6: Cleanup intermediate files (keep current L1 for next iteration)
        print("Step 6/6: Cleaning up intermediate files...")
        
        # List files before cleanup
        before_files = glob.glob(os.path.join(TEMP_L1_DIR, '*.asdf'))
        before_files.extend(glob.glob(os.path.join(TEMP_L1_DIR, '*.parquet')))  # Add parquet files
        
        # Delete all files EXCEPT the current L1 (needed for next iteration's persistence)
        deleted = []
        current_l1_basename = os.path.basename(l1_filepath)
        
        for filepath in before_files:
            basename = os.path.basename(filepath)
            # Keep the current L1 file for next iteration's persistence modeling
            if basename == current_l1_basename:
                print(f"   Keeping {basename} for next iteration's persistence")
                continue
            
            try:
                os.remove(filepath)
                deleted.append(basename)
            except Exception as e:
                print(f"     Could not delete {basename}: {e}")
        
        # Verify cleanup worked
        after_files = glob.glob(os.path.join(TEMP_L1_DIR, '*.asdf'))
        
        if deleted:
            print(f"   Deleted {len(deleted)} files")
            if len(deleted) <= 3:
                for f in deleted:
                    print(f"    - {f}")
            else:
                for f in deleted[:3]:
                    print(f"    - {f}")
                if len(deleted) > 3:
                    print(f"    ... and {len(deleted)-3} more")
        else:
            print(f"   No files deleted!")
            if before_files:
                print(f"   But {len(before_files)} files exist in directory:")
                for f in before_files[:3]:
                    print(f"    - {os.path.basename(f)}")
        
        if after_files:
            print(f"   WARNING: {len(after_files)} files remain after cleanup:")
            for f in after_files[:3]:
                print(f"    - {os.path.basename(f)}")
        else:
            print(f"   Temp directory is clean")
        
        del analyzer, analyzer_cleaned, l2_model
        gc.collect()
        
        # Save checkpoint after EVERY file (safe resume)
        total_processed += 1
        save_checkpoint(file_num, total_processed)
        
        # Only print checkpoint message every 10 files to reduce clutter
        if file_num % 10 == 0:
            print(f"   Checkpoint saved (file {file_num})")
        
        print(f" Completed file {file_num}")
        
    except Exception as e:
        print(f"\n✗ ERROR processing file {file_num}: {e}")
        import traceback
        traceback.print_exc()
        
        save_checkpoint(file_num - 1, total_processed)
        
        if 'l1_filepath' in locals() and os.path.exists(l1_filepath):
            os.remove(l1_filepath)
        if 'l2_filepath' in locals() and os.path.exists(l2_filepath):
            os.remove(l2_filepath)
        
        break  # Stop on error



# ============================================================================
# CREATE UNCERTAINTY SUMMARY
# ============================================================================

def create_uncertainty_summary(photometry_csv, mag):
    """Create uncertainty summary by image step."""
    
    # Load photometry data
    phot_df = pd.read_csv(photometry_csv)
    
    # Group by image_id and calculate statistics
    summary = phot_df.groupby('image_id').agg({
        'id': 'count',
        'flux_jy_err': ['mean', 'std', 'median'],
        'magnitude_err': ['mean', 'std', 'median'],
        'snr': ['mean', 'std', 'median']
    }).reset_index()
    
    # Flatten column names
    summary.columns = [
        'image_id', 
        'n_stars',
        'flux_err_mean', 'flux_err_std', 'flux_err_median',
        'mag_err_mean', 'mag_err_std', 'mag_err_median',
        'snr_mean', 'snr_std', 'snr_median'
    ]
    
    # Save
    output_csv = os.path.join(RESULTS_DIR, f'Uncertainty_Summary_by_Steps_Mag_{mag}.csv')
    summary.to_csv(output_csv, index=False)
    print(f"\n Created uncertainty summary: {output_csv}")
    
    return summary

# Create uncertainty summary
print(f"\nCreating uncertainty summary...")
create_uncertainty_summary(PHOTOMETRY_CSV, MAG)

# Final summary
print(f"\n{'='*80}")
print("PROCESSING COMPLETE!")
print(f"{'='*80}")
print(f"Total files processed: {total_processed}")
print(f"Time: {(time.time() - processing_start_time)/3600:.1f} hrs ({(time.time() - processing_start_time)/86400:.2f} days)")
print(f"\nResults:")
print(f"  L1 metrics: {L1_METRICS_CSV}")
print(f"  Photometry: {PHOTOMETRY_CSV}")
print(f"  Persistence: {PERSISTENCE_CSV}")
print(f"\n Space saved: ~{total_processed * 0.7:.1f} GB")
print(f"{'='*80}")


✓ All imports loaded
✓ Global configuration loaded
✓ Photometric constants loaded
✓ Intermediate data extraction configured
  Jump detection: False
  WCS astrometry: False
  Ramp quality: False
  DQ flags: False
✓ L1 generation function loaded
✓ Helper functions loaded

SPACE-EFFICIENT PIPELINE - FULLY SELF-CONTAINED
Observation period: 72 days
Cadence: 12 minutes (720 seconds)
Total exposures: 8640
Estimated processing time: ~21.0 days
✓ Initialized ./analysis_results/l1_metrics.csv
✓ Initialized ./analysis_results/Photometry_Mag_10.csv
✓ Initialized ./analysis_results/persistence.csv
✓ Using catalog: ./star10.ecsv
✓ Photometry will use auto-detected source positions

STARTING PROCESSING
Total files: 8640


Processing 1/8640 (0.01%)
Time: Day 0.00 (0.0 hrs)
Step 1/6: Generating L1...


2026-01-20 23:02:02 INFO     Simulating filter F146...
2026-01-20 23:03:20 WARNING  max(flat) > 1.1; this seems weird?!
2026-01-20 23:03:20 INFO     Adding 1 sources to image...
2026-01-20 23:03:20 INFO     Rendered 1 sources...
2026-01-20 23:03:23 INFO     Apportioning electrons to resultants...
2026-01-20 23:04:10 INFO     Adding IPC...
2026-01-20 23:04:11 INFO     Adding read noise...
2026-01-20 23:04:15 INFO     Simulation complete.


  ✓ Generated: roman_sim_F146_WFI01_L1_1.asdf
Step 2/6: Analyzing L1 and replacing saturated pixels...
Loading L1 file: ./temp_l1_processing/roman_sim_F146_WFI01_L1_1.asdf
Found MA table read pattern with 5 resultants
  Resultant 0: frames [1], time = 3.16s
  Resultant 1: frames [2, 3], time = 9.49s
  Resultant 2: frames [4, 5, 6, 7, 8, 9], time = 28.46s
  Resultant 3: frames [10, 11, 12, 13, 14, 15, 16, 17], time = 53.76s
  Resultant 4: frames [18], time = 56.92s
  MA table: 18 frames, 56.92s total
Loaded ramp data: (5, 4096, 4096)
MA table 3: 5 resultants
Extracting individual resultants with MA table context...
  resultant_00: time=3.16s, frames_avg=1, mean=45.5 DN, median=13.0 DN
  resultant_01: time=9.49s, frames_avg=2, mean=52.4 DN, median=22.0 DN
  resultant_02: time=28.46s, frames_avg=6, mean=74.6 DN, median=45.0 DN
  resultant_03: time=53.76s, frames_avg=8, mean=117.0 DN, median=87.0 DN
  resultant_04: time=56.92s, frames_avg=1, mean=144.4 DN, median=114.0 DN
Extracted 5 resul

2026-01-20 23:04:22 INFO     ExposurePipeline instance created.
2026-01-20 23:04:22 INFO     DQInitStep instance created.
2026-01-20 23:04:22 INFO     SaturationStep instance created.
2026-01-20 23:04:22 INFO     RefPixStep instance created.
2026-01-20 23:04:22 INFO     LinearityStep instance created.
2026-01-20 23:04:22 INFO     DarkCurrentStep instance created.
2026-01-20 23:04:22 INFO     RampFitStep instance created.
2026-01-20 23:04:22 INFO     AssignWcsStep instance created.
2026-01-20 23:04:22 INFO     FlatFieldStep instance created.
2026-01-20 23:04:22 INFO     PhotomStep instance created.
2026-01-20 23:04:22 INFO     SourceCatalogStep instance created.
2026-01-20 23:04:22 INFO     TweakRegStep instance created.
2026-01-20 23:04:22 INFO     Starting Roman exposure calibration pipeline ...
2026-01-20 23:04:22 INFO     Step dq_init running with args (<roman_datamodels.datamodels._datamodels.ScienceRawModel object at 0x7f6a46f2c7a0>,).
2026-01-20 23:04:22 INFO     Step dq_init don

  ✓ Generated: roman_sim_F146_WFI01_L2_cal_1.asdf
Step 4/6: Photometry (saturated pixels already removed in L1)...
  Saturation (from L2 DQ): 7255 core, 24712 total
  ✓ Saved reference image
  Auto-detected source: (1997, 2003)
  Peak value: 5558.19 DN
  ✓ Using detected position for photometry
  DEBUG: source_positions = [(np.int64(2003), np.int64(1997))]
  DEBUG: About to run photometry loop for 1 sources...
  DEBUG: Processing source 1 at (2003, 1997)
  DEBUG: Saved photometry result to ./analysis_results/Photometry_Mag_10.csv
  ✓ Measured 1 sources
  DEBUG: reference_image is set
Step 5/6: Detecting persistence...
  ✓ Persistence: 0.0000%
Step 6/6: Cleaning up intermediate files...
  ✓ Keeping roman_sim_F146_WFI01_L1_1.asdf for next iteration's persistence
  ✓ Deleted 2 files
    - roman_sim_F146_WFI01_L1_1_unsat.asdf
    - roman_sim_F146_WFI01_L2_cal_1.asdf
  ⚠ WARNING: 1 files remain after cleanup:
    - roman_sim_F146_WFI01_L1_1.asdf
✓ Completed file 1

Processing 2/8640 (0.02%)

...

2026-01-21 04:10:10 INFO     Simulating filter F146...
2026-01-21 04:11:27 WARNING  max(flat) > 1.1; this seems weird?!
2026-01-21 04:11:27 INFO     Adding 1 sources to image...
2026-01-21 04:11:27 INFO     Rendered 1 sources...
2026-01-21 04:11:30 INFO     Apportioning electrons to resultants...
2026-01-21 04:12:19 INFO     Adding IPC...
2026-01-21 04:12:20 INFO     Adding read noise...
2026-01-21 04:12:24 INFO     Simulation complete.


  ✓ Generated: roman_sim_F146_WFI01_L1_106.asdf
Step 2/6: Analyzing L1 and replacing saturated pixels...
Loading L1 file: ./temp_l1_processing/roman_sim_F146_WFI01_L1_106.asdf
Found MA table read pattern with 5 resultants
  Resultant 0: frames [1], time = 3.16s
  Resultant 1: frames [2, 3], time = 9.49s
  Resultant 2: frames [4, 5, 6, 7, 8, 9], time = 28.46s
  Resultant 3: frames [10, 11, 12, 13, 14, 15, 16, 17], time = 53.76s
  Resultant 4: frames [18], time = 56.92s
  MA table: 18 frames, 56.92s total
Loaded ramp data: (5, 4096, 4096)
MA table 3: 5 resultants
Extracting individual resultants with MA table context...
  resultant_00: time=3.16s, frames_avg=1, mean=45.5 DN, median=13.0 DN
  resultant_01: time=9.49s, frames_avg=2, mean=52.4 DN, median=21.0 DN
  resultant_02: time=28.46s, frames_avg=6, mean=74.6 DN, median=45.0 DN
  resultant_03: time=53.76s, frames_avg=8, mean=116.8 DN, median=87.0 DN
  resultant_04: time=56.92s, frames_avg=1, mean=144.2 DN, median=114.0 DN
Extracted 5 r

2026-01-21 04:12:30 INFO     ExposurePipeline instance created.
2026-01-21 04:12:30 INFO     DQInitStep instance created.
2026-01-21 04:12:30 INFO     SaturationStep instance created.
2026-01-21 04:12:30 INFO     RefPixStep instance created.
2026-01-21 04:12:30 INFO     LinearityStep instance created.
2026-01-21 04:12:30 INFO     DarkCurrentStep instance created.
2026-01-21 04:12:31 INFO     RampFitStep instance created.
2026-01-21 04:12:31 INFO     AssignWcsStep instance created.
2026-01-21 04:12:31 INFO     FlatFieldStep instance created.
2026-01-21 04:12:31 INFO     PhotomStep instance created.
2026-01-21 04:12:31 INFO     SourceCatalogStep instance created.
2026-01-21 04:12:31 INFO     TweakRegStep instance created.
2026-01-21 04:12:31 INFO     Starting Roman exposure calibration pipeline ...
2026-01-21 04:12:31 INFO     Step dq_init running with args (<roman_datamodels.datamodels._datamodels.ScienceRawModel object at 0x7f6a61c03230>,).
2026-01-21 04:12:31 INFO     Step dq_init don

  ✓ Generated: roman_sim_F146_WFI01_L2_cal_106.asdf
Step 4/6: Photometry (saturated pixels already removed in L1)...
  Saturation (from L2 DQ): 7255 core, 24719 total
  Auto-detected source: (1997, 2003)
  Peak value: 5581.07 DN
  ✓ Using detected position for photometry
  DEBUG: source_positions = [(np.int64(2003), np.int64(1997))]
  DEBUG: About to run photometry loop for 1 sources...
  DEBUG: Processing source 1 at (2003, 1997)
  DEBUG: Saved photometry result to ./analysis_results/Photometry_Mag_10.csv
  ✓ Measured 1 sources
  DEBUG: reference_image is set
Step 5/6: Detecting persistence...
  ✓ Persistence: 0.0005%
Step 6/6: Cleaning up intermediate files...
  ✓ Keeping roman_sim_F146_WFI01_L1_106.asdf for next iteration's persistence
  ✓ Deleted 3 files
    - roman_sim_F146_WFI01_L1_105.asdf
    - roman_sim_F146_WFI01_L2_cal_106.asdf
    - roman_sim_F146_WFI01_L1_106_unsat.asdf
  ⚠ WARNING: 1 files remain after cleanup:
    - roman_sim_F146_WFI01_L1_106.asdf
✓ Completed file 106



2026-01-21 04:13:10 INFO     Simulating filter F146...
2026-01-21 04:14:26 WARNING  max(flat) > 1.1; this seems weird?!
2026-01-21 04:14:26 INFO     Adding 1 sources to image...
2026-01-21 04:14:26 INFO     Rendered 1 sources...
2026-01-21 04:14:29 INFO     Apportioning electrons to resultants...
